# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [28]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [29]:
ETtoday_url = 'https://www.ettoday.net/news/focus/%E5%9C%8B%E9%9A%9B/'  #國際新聞


### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [30]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
while True:
     # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break
    last_height = new_height
    #提示：可參考以下的Stack Overflow: 
    #https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    
print("bottom of the web")

bottom of the web


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [12]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [31]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 7-11「超創新徵才海報」瘋傳！　10萬網友讚嘆：平面設計的先驅...

實習記者賴韻如／綜合報導
不少企業都非常歡迎沒有經驗的社會新鮮人來應徵工讀生，還會製作徵才廣告來吸引大家注意力。日本一名網友近日就分享自己在街上看見這張超創新的「711徵才海報」，不僅簡單明瞭，大家都能輕輕鬆鬆了解文案內容，照片曝光後讓不少網友大讚「真的太強！」
原PO「@nkgw73_」於本月3日在推特中分享了這張位於東京澀谷區一間7-11便利商店外貼的招募工讀生海報，只見海報上是一個大大的Line聊天室截圖，內容則是店長請某位員工幫忙製作徵人海報的對話，「謝謝你幫忙做新商品的海報，做得很棒！想再請你製作徵才的海報。」員工緊接著詢問徵才細節，店長則分別回應了3個時段及時薪，最後附上自己的電話、姓名。
這張照片一曝光後，立即吸引網友的注意力，短短時間就吸引10萬網友按讚，大家紛紛讚嘆這個設計實在太強，不僅有創意又簡單明瞭，「這海報閱讀起來比普通海報更有趣！」、「好可愛的海報」、「封他為海報大師」、「什麼？這是一張徵才海報？我現在才看出來」、「太新穎了」、「根本平面設計的先驅」、「非常醒目、易於閱讀、設計精良」、「能用這樣的方式徵才，感覺上班氣氛很好」。
----------------------------------------------------------------------

[1] 驚悚預言第三次世界大戰！川普賭上政治生命擊斃蘇雷曼尼　專家曝關鍵：尾巴搖狗

圖文／鏡週刊
伊朗今（8）日對伊拉克兩座有美軍駐守的基地發射了超過12枚地對地飛彈，稱「慘烈的報復」剛剛展開，引發美伊恐將開戰的疑慮。
事實上，在美國總統川普3日下令以無人機狙殺伊朗將領蘇雷曼尼（Qasem Soleimani）後，幾小時之內，推特上立即出現「第三次世界大戰」的驚悚預言；社群媒體上另一個聲量很大的說法，則是「尾巴搖狗（Wag the Dog）」，暗指川普是為了轉移國內對彈劾案的注意力，才做出這個風險極大的決定。
▲一架烏克蘭客機從伊朗起飛後不久墜毀。
常有人說，關於川普最可以被預測的一點，就是他的不可預測性。
川普振振有詞，說蘇雷曼尼正在研擬「邪惡」計畫

皇室發送聖誕賀卡已有一個世紀以上的悠久歷史，1914年，當時瑪麗公主的聖誕賀卡，是對士兵們送上的禮物的一部分；1942年，當時還是公主的伊莉莎白二世曾發送穿著戰時制服給軍隊們祝福；1987年的則是當時還在世的黛安娜王妃以及她的兩個兒子威廉和哈利。每一張卡片都吐露著佳節的祝福，也是一段又一段的歷史回憶。
查爾斯王子在2005年的聖誕賀卡上，是來自一張他與Camilla Parker Bowles的婚禮照片。
完整圖輯請點此
----------------------------------------------------------------------

[11] 等了7年的正義！印度女大生公車上遭性侵殺害　4惡徒2週後絞死

▲公車姦殺案4名惡徒，歷經7年訴訟後終於要在本月22日伏法。（翻自news.sky.com）
圖文／鏡週刊
轟動印度社會的2012年公車姦殺案，當時引發民眾大規模上街抗議及各國譴責，受害者家屬終於在等了7年後得到一個公道正義，新德里法院昨（7日）駁回4名犯人的上訴，簽署處死令，宣判於22日上午將4人絞死。
這起公車姦殺案件發生於2012年12月中旬，當時一名23歲醫學院物理治療系女大生辛格潘迪（Jyoti Singh Pandey），晚間和男性友人看完電影準備返家時，在新德里搭上一輛公車，卻沒想到遇上6名惡徒，辛格的男性友人試圖阻止卻遭圍毆並被擊昏，而辛格則被6人拖到公車後排性侵，事後還將2名受害者從行進中的公車上拋飛，辛格潘迪則是傷重不治身亡。
此案引發大批示威者多日在印度各大城市抗議，要求政府重視女性安全和性侵問題。印度警方迅速逮捕6名惡徒，其中公車司機在獄中上吊身亡，而1人因當時尚未成年被判感化3年，逃過死刑制裁；另外4名全被判處死刑，上月最高法院駁回最後一份免死上訴，於7日宣布執行死刑日期，訂於22日上午7時處以絞刑。
★鏡週刊關心您，再給自己一次機會
更多鏡週刊報導
----------------------------------------------------------------------

[12] 45分鐘殺19人！　安養院殺人魔首次開庭「手塞嘴巴＋全身扭曲」被迫中止

▲日本安養院殺人魔植松聖。（圖／CFP）
實習記者賴韻如／綜合報導
日本2016年7月發生一起嚴重凶殺案，凶手植松聖持刀闖入神奈川縣「津

▲33歲奧地利總理庫爾茨。（圖／路透）
記者康心禹／綜合報導
奧地利保守派「人民黨」（OeVP）黨魁庫爾茲（Sebastian Kurz）7日宣誓就任總理，33歲的他成為全球最年輕總理，誓言「捍衛氣候與邊界」，他先前與極右翼政治勢力合組的聯合政府瓦解之後，這次與綠黨組成了一個全新的、截然不同的聯盟。
綜合《以色列時報》、《德國之聲》報導，新一屆內閣是奧地利史上首次由人民黨和綠黨聯合執政，也是綠黨首次在全國層面參與執政。此次的內閣成員17人裡面，有9名（過半數）為女性，是奧地利史上第一次女性部長多於男性，其中包括第一位女國防部長克勞迪婭·坦納（Klaudia Tanner）和綠黨司法部長阿爾瑪·扎迪奇（Alma Zadic）。
▲庫爾茨宣誓就職。（圖／路透）
根據協議內容，庫爾茨與綠黨領袖克格勒（Werner Kogler）會分別成為總理及副總理，而在17名內閣部長職位中，綠黨獲分配環境、交通、基建和能源部門首長職位，庫爾茨的人民黨則主管財政、內政、國防和外交事務。
新政府除了樹立在2040年前實現碳中立的環境目標外，亦將在移民和難民議題上推行更為嚴厲的政策。
在2019年9月提前舉行的國會改選中，人民黨獲勝成為國民議會第一大黨。 11月11日庫爾茨宣布選擇與綠黨進行組閣談判。2020年1月1日，人民黨和綠黨完成組閣談判，就聯合執政協議達成一致，共同組建新一屆奧地利政府。
▲庫爾茨與內閣成員。（圖／路透）
----------------------------------------------------------------------

[23] 時隔33年！沖繩爆發豬瘟　當地撲殺1813頭豬、特有種「阿古豬」也在列

▲沖繩爆發豬瘟。（圖／路透社）
記者蔡玟君／綜合報導
日本沖繩爆出豬瘟疫情！沖繩縣農林水產部8日表示，宇流麻市2間養豬場的豬群確定感染豬瘟，因此決定撲殺包括2間養豬場在內，共3間養豬場、計1813頭豬隻，也是沖繩時隔33年再傳豬瘟。
綜合日本媒體報導，沖繩縣農林水產部證實，宇流麻市飼養約400隻豬的養豬場，在6日發生數頭隻咳嗽病死亡的情形，經過送驗呈現陽性反應，確定感染豬瘟。
▲沖繩撲殺1813頭豬。（圖／路透社）
日本本土2018年9月已爆發豬瘟，目前有10個地方為疫區，此次沖繩自1986年後再傳豬瘟，當地政府表示將撲殺1813頭豬，其中包

在伊朗空襲美軍伊拉克的消息震驚全球之際，西南部城市當地8日清晨接連發生規模4.9、規模4.5兩起地震，根據美國地質調查局最新數據，兩起地震深度都只有10公里，均屬極淺層地震。此外，震央地點附近就是核電廠。
數據顯示，規模4.9地震先在清晨5時50分來襲，震央位在博拉茲詹（Borazjan）西南方約21公里處，緊接著規模4.5地震於6時49分發生，震央位在博拉茲詹南方約17公里處。
根據華盛頓時報（The Washington Times）報導，首次規模4.9地震的震央距離Bushehr核電廠約40英里（約64公里）處。當局已證實，此次地震實屬自然現象，與空襲事件並無關係。
----------------------------------------------------------------------

[35] 獵豹睡到流口水還「牽絲」滴到地上　醒來超呆萌：我做什麼了嗎

記者李振慧／綜合報導
平時看起來兇猛的獵豹竟然也有呆萌的一面，日本廣島市一間動物園的一隻獵豹近來被拍到，熟睡的牠竟然睡到不小心流口水，還「牽絲」滴到地上，模樣十分可愛。
這支可愛影片近來被廣島市安佐動物園分享在推特上，影片中可看到，一隻正在閉目養神的獵豹竟然睡到流出口水，還「牽絲」懸掛在半空中，最後不敵地心引力滴了下來，獵豹醒來後緊張地四處張望，彷彿在擔心自己剛剛的糗樣會被人看到。
影片自上月28日分享至推特後，獲得3.2萬人次觀看，許多網友紛紛被獵豹呆萌的模樣逗笑，「太可愛了」、「拍得好，竟然能拍到這幕」、「拍攝的工作人員一定笑得很開心，獵豹注意到了，才會一臉『我做了什麼嗎？』的樣子」、「我想知道牠是做了什麼好夢」。
----------------------------------------------------------------------

[36] 伊朗憤怒不熄！高官PO川普「900億房地產清單」　暗示恐成攻擊目標

▲紐約川普大樓（Trump Tower）。（圖／達志影像／美聯社）
記者陳亭伃／綜合報導
美國總統川普日前空襲巴格達國際機場，擊殺伊朗精神領袖之一的蘇萊曼尼，不僅引發國際震驚，也讓伊朗國內上下相當不滿。伊朗社群媒體上也出現要懸賞8000萬美元（約台幣24億元）要川普人頭；此外，更有伊朗高官貼出川普資產的相關消息，外界揣測恐怕將成為攻擊的目標之一。
綜

▲女子可能為了自拍而失去一條手臂。（圖／示意圖，與本新聞無關／達志影像）
記者李振慧／綜合報導
墨西哥一動物園近來發生美洲豹咬人事件，一名女遊客為了拍出自拍美照，在過程中不小心與美洲豹靠得太近，導致右手臂被咬傷並劃傷了臉，幸好被其他遊客緊急從籠中拉出才脫身，由於受傷情況嚴重，很有可能會讓她因為自拍而面臨截肢下場。
事件發生在墨西哥中部韋拉克魯斯州科爾多瓦市(Córdoba)的Zoologico Bio動物園，發言人迪亞茲(Gonzalo Rodriguez Diaz)表示，30歲女遊客弗拉格薩(Noemi Rosas Fragoza)為了和美洲豹自拍，擅自跨越警戒線，導致距離太近被美洲豹攻擊咬傷。
►欸~妳過來一下！妳過來一下
----------------------------------------------------------------------

[49] 影／美國伊朗恩怨「一次看懶人包」　空襲復仇起源過程…牽整個國際局勢！

記者陳亭伃／綜合報導
自上月31日美軍空襲伊拉克巴格達國際機場後，美國、伊朗、伊拉克三方進入緊繃狀態，事發起源經過，帶您一次了解：
從一個承包商的死，演變成國際局勢動盪
起因自什葉派民兵（伊朗革命衛隊支持）對美軍駐守伊拉克基地進行突襲，其中包括上月27日美國民用承包商遭到火箭襲擊身亡，美方表示皆為蘇萊曼尼所策畫。上月29日美軍對伊拉克親伊朗民兵空襲；緊接上月31日美國駐伊拉克大使館遭到上萬民眾包圍攻擊，甚至一度強衝進館內的緩衝接待區，現場不僅放火燒掉監視塔、還攻破管制中心。當時美國總統川普在推特上表示：「倘若造成美方人員傷亡，將會不計一切代價反擊」。
雙方認知不同，怒火觸動戰火僅一線之隔
根據美方調查，認為31日的大使館攻擊事件明顯為「伊朗政府」煽動，在遭到入侵的第一時間，大使館就與美國防部連線，並且立即派出阿帕契直升機盤旋示警，接著先派出500士兵抵達科威特，接著又集結3500人的反應部隊「視狀況決定是否進入伊拉克」，這時外界尚未發現美方準備復仇。
觸動三方怒火的空襲，美、伊朗、伊拉克全面進入戒備狀態
美駐伊拉克大使館遭到攻擊，讓川普心生憤怒，加上進入2020連任之路，對於美軍、美國人民的生命安全需有明顯作為。美軍在巴格達時間3日清晨，以無人機空襲，當場擊斃正準備自巴格達機場離開的伊朗高階將領「聖城旅」（Quds Fo

▲伊朗向美軍發射飛彈攻擊。（圖／翻攝自推特）
記者詹雅婷／綜合報導
在伊朗空襲美國駐伊拉克基地的消息傳出後，美國聯邦航空總署（FAA）已於7日發出通知，限制美國非軍事飛機飛越伊拉克、伊朗、波斯灣、阿曼灣上空。此外，新加坡航空公司（Singapore Airlines）已證實，所有飛越伊朗領空的航班全數改道，但並未給出具體理由。
綜合CNN、路透報導，在中東戰事一觸即發之際，目前美國當局已對非軍事飛機做出限制。根據美國聯邦航空總署的聲明，目前民營行公司的飛行限制區域為伊拉克、伊朗、波斯灣、阿曼灣上空，「FAA將會密切關注中東局勢發展，繼續與我們的國家安全夥伴協調，並與美國、國外的航空公司分享最新資訊。」
----------------------------------------------------------------------

[61] 影／美國防部長：不和伊朗開戰但「準備好結束戰爭」　否認自伊拉克撤軍

▲ 艾斯培不後悔向川普提及殺害蘇萊曼尼的選項。（圖／達志影像／美聯社）
記者陳宛貞／綜合外電報導
伊朗為替喪命的革命衛隊精銳聖城軍指揮官蘇萊曼尼（Qassem Soleimani）復仇，8日清晨對伊拉克境內美軍基地發動攻擊2波飛彈攻擊，更揚言瞄準美國本土。美國國防部長艾斯培（Mark Esper）當地時間7日稍早針對與伊朗情勢表示，不尋求與伊朗開戰，但美國將「準備好結束戰爭」，且在伊拉克議會周末投票表決是否驅逐美軍以前，美國不會主動從伊拉克撤軍。
美國官員聲稱，美軍狙擊蘇萊曼尼是為防止該地區「迫在眉睫」的襲擊發生，可能危及美國公民性命，但至今未提供有關這類襲擊的證據。艾斯培7日接受CNN訪問時也重申，美國握有能夠證明殺害蘇萊曼尼行動之正當性的證據，「我可以保證，這非常有說服力。」
艾斯培說明，蘇萊曼尼被逮個正著，「一個恐怖組織和另一個恐怖組織領導人會面，計畫對美國外交官、軍隊或設施發動更多襲擊。我認為，我們採取了正確的行動，將這些玩家趕出戰場。」當被問及是否後悔向川普提出殺害蘇萊曼尼這個極端選擇時，艾斯培堅稱，「如今是時候將恐怖組織老大趕出戰場了。」
美國今日證實，伊拉克境內2處美軍基地遭到飛彈攻擊。根據伊拉克安全人員向CNN透露的最新訊息，伊朗向美軍駐伊拉克「阿薩德空軍基地」（Ain al-Assad airbase）發射13枚飛彈。伊朗約於當地

▲美伊持續緊張，美股三大指數下跌做收。（圖／路透）
記者錢玉紘／綜合報導
美國與伊朗情勢持續緊張，造成股市收跌。美股7日收盤後，道瓊指數下跌119點或0.42%，收報28,583點；標普指數小跌9點或0.28%收在3,237點；那斯達克指數跌2點或0.03%，收在9,068點。
個股方面，金融服務公司「Cowen」認為，晶片市場有回溫的潛力，美光科技股價上漲了8%，西部數據（Western Digital）則上漲6.8％。另外，瑞銀（UBS）分析師將美國銀行（Bank of America）和摩根大通（J.P. Morgan Chase）的等級調降，兩間公司股票分別跌0.7％和1.7％。
經濟數據方面，美國商務部公布，去年11月的貿易赤字下跌8.2%到431億美元，是2016年10月以來最低，有助於提升美國第4季的經濟成長率，也讓股市跌幅稍微縮減。
伊朗最高國家安全委員會7日揚言，已經擬定13套劇本，任何一項劇本都是美國人的夢魘，而美國軍方官員透露，美方預期伊朗在一、兩天內就會採取行動，有可能鎖定美方指揮官「以牙還牙」，據傳五角大廈已經派遣6架B-52轟炸機前往印度洋的迪亞哥加西亞島（Diego Garcia），距離伊朗4800公里，可以隨時飛往伊朗實施軍事行動。
----------------------------------------------------------------------

[75] 美軍最機動陸軍「第82空降師」部署中東　阿兵哥：要打仗了

記者林彥臣／綜合報導
美國與伊朗的緊張情勢一觸即發，美軍派遣機動性最高的部隊「第82空降師」（82nd Airborne Division）緊急開赴中東，北卡羅來納州（North Carolina）布拉格堡基地（Fort Bragg）有600官兵週末開拔，多數是首次出任務的年輕士兵，他們檢整好步槍與彈藥，並且與家人通過離開美國前的最後一通電話，之後就交出自己的手機。
根據《路透社》報導，布拉格堡的600名大兵，只是第82空降師奉令前往中東約3500名傘兵的一部分，其中大部分士兵的第一站是科威特，最後目的地則列為機密。
其中一位阿兵哥還高喊，「我們要打仗了！兄弟們！」但是大約30~40歲左右的老兵則顯得神情嚴肅，他們可能看過自己的同袍，在戰爭中傷殘，甚至是為國捐軀的畫面。
一名來自維吉尼亞州阿什伯

▲前NFL明星四分衛卡佩尼克(中)，曾發起抗議種族政策的拒唱國歌行動。（圖／達志影像）
記者范姜士航／綜合報導
前NFL明星四分衛卡佩尼克（Colin Kaepernick）在個人Twitter上連續發文抨擊美軍炸死伊朗高階將領蘇萊曼尼（Qassim Suleimani）是「恐怖分子的行為」而引發眾怒，卡佩尼克曾在2016年美式足球賽前播放美國國歌時，以單膝下跪的方式表達對於種族主義的抗議，獲得眾多球星的支持成為美國的話題人物。
過去卡佩尼克支持自由人權，讓他在2019年12月與NIKE共同推出的聯名鞋款大賣，但在前幾日因為針對美軍攻擊伊朗的文章，讓他的形象直轉而下，他在Twitter中寫道「這已經不是什麼新消息了，美國為了帝國主義對黑色、棕色皮膚的人進行恐怖攻擊」、「美國不論在國內及國外都是如此對待黑色或棕色皮膚，美國軍國主義就是美國帝國主義的武器，只針對非白人的國家進行管制及掠奪。」
There is nothing new about American terrorist attacks against Black and Brown people for the expansion of American imperialism.
在這兩則貼文發出後，馬上引起美國保守派的反彈，參議員葛蘭姆（Lindsey Graham）就抨擊「這件事情不是種族主義更不是美國人的問題，這是伊朗的問題，伊朗製造的武器已經造成600位美國人在伊拉克死亡。」而女演員明迪.羅賓森（Mindy Robinson）則表示，「我們正在對抗恐怖份子，如果你要捍衛伊斯蘭的恐怖主義，你可以移民中東地區」。
We’re killing terrorists, bud. I know you’re a spoiled POS that hates America and all...but if you’re seriously going to defend Islamic terrorism please feel free to move to the Middle East where I’m sure you can experience um, whatever you think you’re not finding here.
針對中東目前不斷擴大的衝突，已讓美國國內掀起撤軍、反戰的

酸民當老大
二○○八年十一月四日，美國邁入了新紀元；至少，當晚我們如此深信。史上頭一遭，美國選出了一位非裔總統入主白宮。深深烙印這偉大國家歷史的撕裂傷似乎終於癒合，民意調查也眾口一詞，「再也沒有種族主義了。」就算在南方最守舊的州，我們也從此找不到任何一位選民承認膚色會影響他的投票考量。花費了三百多年的光陰，美國這個大熔爐似乎終於水到渠成。
▲美國總統川普（Donald Trump）。（圖／達志影像／美聯社）
然而，同一天、同個夜晚，巴拉克．歐巴馬（Barack Obama）在芝加哥的格蘭特公園慶祝他的勝選；與格蘭特公園大螢幕相距遙遠的所在，截然不同的現象發生了。Google的數據與民調完全相反，揭露了人們真實的想法與行為：在歐巴馬的勝選之夜，某些州在網路上搜尋「第一個黑鬼總統」（first nigger president）的次數超過了「第一位黑人總統」（first black president）。也是在同一個夜晚，種族主義社群網站Stormfront創下了破紀錄的註冊人潮。
再說得具體些，官方歷史宣布種族仇恨已經終結的這一刻，種族仇恨卻正在重新組織，並採用更隱微、更當代的嶄新形式，讓它能在八年之後轟動武林、強勢回歸。當時，所有人對此渾然不覺，可能只有超越傳統政治人物的第六感預知者才能察覺此一現象。從歐巴馬走馬上任最初的幾個月，一切就動起來了，這位有實力動員隱蔽但數量巨大的種族主義選民的預知者，化身為一張最不可能的面孔──唐納．川普。
川普自己就是網路引戰者。歐巴馬出生證明的爭議就已經是一種網路引戰，川普的官方競選活動也依循一樣的理路。二○一五年六月，他忽然空降總統選戰，做了兩樣恐怕會讓任何一個總統競選活動直接畫上句點的選舉操作。首先，他以一段演講宣告他正式投入共和黨總統初選。該演講很顯然是即興發揮；演講中，他大力排斥墨西哥移民，甚至形容他們為「強暴犯」。接著，他在幾天之後發表對共和黨參議員約翰．馬侃（John McCain）的看法。馬侃是美國政界貨真價實的元老級人物，憲政光譜的每一端都敬重他。川普說，「他不是戰爭英雄。因為他被俘虜過。我喜歡沒被俘虜的人。」
好，先不談墨西哥人。一個共和黨的總統參選人，用網路引戰的方式，嗆馬侃和戰爭老兵？這在「大老黨」（Grand Old Party，美國共和黨的別稱）陣營裡前所未見。強烈的憤慨立刻排山倒海而來。共和黨全國委員會戰